In [11]:
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | tee /etc/apt/sources.list.d/coral-edgetpu.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 6A030B21BA07F4FB

!apt update
!apt install edgetpu -y

deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Executing: /tmp/tmp.uhO1gEJ4Kc/gpg.1.sh --keyserver
keyserver.ubuntu.com
--recv-keys
6A030B21BA07F4FB
gpg: requesting key BA07F4FB from hkp server keyserver.ubuntu.com
gpg: key BA07F4FB: "Google Cloud Packages Automatic Signing Key <gc-team@google.com>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Get:1 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  InRelease
Ign:1 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  InRelease
Get:2 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  Release [574 B]
Get:2 file:/var/nvinfer-runtime-trt-repo-5.0.2-ga-cuda10.0  Release [574 B]0m
Hit:3 http://archive.ubuntu.com/ubuntu xenial InRelease                        
Hit:4 http://archive.ubuntu.com/ubuntu xenial-updates InRelease                
Hit:6 http://archive.ubuntu.com/ubuntu xenial-backports InRelease              3m33m
Hit:7 http://security.ubuntu.com/ubuntu xenial-security InRelease

In [12]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.13.1


In [13]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

In [14]:
def build_keras_model():
    return keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation='softmax')
    ])

In [15]:
# train
train_graph = tf.Graph()
train_sess = tf.Session(graph=train_graph)

keras.backend.set_session(train_sess)
with train_graph.as_default():
    train_model = build_keras_model()
    
    tf.contrib.quantize.create_training_graph(input_graph=train_graph, quant_delay=100)
    train_sess.run(tf.global_variables_initializer())    

    train_model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    train_model.fit(train_images, train_labels, epochs=5)
    
    # save graph and checkpoints
    saver = tf.train.Saver()
    saver.save(train_sess, 'checkpoints')

INFO:tensorflow:Inserting fake quant op activation_Mul_quant after batch_normalization_v1/batchnorm/mul_1
INFO:tensorflow:Inserting fake quant op activation_Add_quant after batch_normalization_v1/batchnorm/add_1
Epoch 1/5
60000/60000 [==============================] - 5s 82us/sample - loss: 0.4841 - acc: 0.8301
Epoch 2/5
60000/60000 [==============================] - 5s 80us/sample - loss: 0.4031 - acc: 0.8572
Epoch 3/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.3731 - acc: 0.8659
Epoch 4/5
60000/60000 [==============================] - 5s 83us/sample - loss: 0.3533 - acc: 0.8732
Epoch 5/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3393 - acc: 0.8750


In [16]:
# eval
eval_graph = tf.Graph()
eval_sess = tf.Session(graph=eval_graph)

keras.backend.set_session(eval_sess)

with eval_graph.as_default():
    keras.backend.set_learning_phase(0)
    eval_model = build_keras_model()
    tf.contrib.quantize.create_eval_graph(input_graph=eval_graph)
    eval_graph_def = eval_graph.as_graph_def()
    saver = tf.train.Saver()
    saver.restore(eval_sess, 'checkpoints')

    frozen_graph_def = tf.graph_util.convert_variables_to_constants(
        eval_sess,
        eval_graph_def,
        [eval_model.output.op.name]
    )

    with open('/tf/ml-quantization-test/frozen_model.pb', 'wb') as f:
        f.write(frozen_graph_def.SerializeToString())

INFO:tensorflow:Inserting fake quant op activation_Mul_quant after batch_normalization_v1/batchnorm/mul_1
INFO:tensorflow:Inserting fake quant op activation_Add_quant after batch_normalization_v1/batchnorm/add_1
INFO:tensorflow:Restoring parameters from checkpoints
INFO:tensorflow:Froze 20 variables.
INFO:tensorflow:Converted 20 variables to const ops.


In [17]:
!tflite_convert \
    --output_file=/tf/ml-quantization-test/model.tflite \
    --graph_def_file=/tf/ml-quantization-test/frozen_model.pb \
    --inference_type=QUANTIZED_UINT8 \
    --input_arrays=flatten_input \
    --output_arrays=dense_1/Softmax \
    --mean_values=0 \
    --std_dev_values=255

2019-06-30 16:16:22.528616: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-06-30 16:16:22.558682: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-06-30 16:16:22.559096: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x4635b90 executing computations on platform CUDA. Devices:
2019-06-30 16:16:22.559112: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): GeForce GTX 980 Ti, Compute Capability 5.2
2019-06-30 16:16:22.580950: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3600000000 Hz
2019-06-30 16:16:22.581833: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x469fe10 executing computations on platform Host. Devices:
2019-06-30 16:16:22.581847: I tensorflow/

In [18]:
!edgetpu_compiler /tf/ml-quantization-test/model.tflite

Edge TPU Compiler version 1.0.249710469
INFO: Initialized TensorFlow Lite runtime.

Model compiled successfully in 11 ms.

Input model: /tf/ml-quantization-test/model.tflite
Input size: 102.27KiB
Output model: model_edgetpu.tflite
Output size: 152.55KiB
On-chip memory available for caching model parameters: 8.01MiB
On-chip memory used for caching model parameters: 106.75KiB
Off-chip memory used for streaming uncached model parameters: 4.00KiB
Number of Edge TPU subgraphs: 1
Total number of operations: 5
Operation log: model_edgetpu.log
See the operation log file for individual operation details.
